# Customizing Our Agent

In this exercise, you'll customize both your agent and its tool to improve **clarity**, **transparency**, and **structure**.

![Customizing your agent](images/agent_transparency.png)

### ❗️ Note: Run the **hidden cell** below to initialize the agent, before running the rest of the code. ❗️ 

In [1]:
!pip install -q haystack-ai


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


### 🛠️ Customizing the Tool

Override the tool's name and description by passing them directly to the `ComponentTool`.
Use `outputs_to_string` to format the list of documents before they're passed to the LLM, and `outputs_to_state` to store them in the agent’s internal state.

In [2]:
from haystack.tools import ComponentTool
from haystack.components.websearch import SerperDevWebSearch

def doc_to_string(documents) -> str:
    result_str = ""
    for document in documents:
        result_str += f"Content for {document.meta['link']}: {document.content}\n\n"
    return result_str

search_tool = ComponentTool(
    component=SerperDevWebSearch(top_k=5),
    name="web_search_tool",
    description="Search the web",
    outputs_to_string={"source": "documents", "handler": doc_to_string}, 
    outputs_to_state={"documents": {"source": "documents"}}
)

### ⚙️ Configuring a Traceable Agent
Define `state_schema` to store the retrieved documents.
Use the built-in `print_streaming_chunk` callback to display tool calls and their results as the agent runs.

In [3]:
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage, Document

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4o-mini"),
    system_prompt="""
    You are a helpful AI assistant that has access to internet.
    Keep your answer concise and use the tools that you're provided with to answer the user's questions.
    """,
    tools=[search_tool],
    state_schema={"documents":{"type":list[Document]}}, 
    streaming_callback=print_streaming_chunk 
)

### ▶️ Running the Agent with Trace Logs
Run the agent with a query and watch the full execution trace:
tool selection, query parameters, document results, and the final LLM response.

In [4]:
agent_results = agent.run(messages=[ChatMessage.from_user("What are the common side effects of hyaluronic acid?")])

[TOOL CALL]
Tool: web_search_tool 
Arguments: {"query":"common side effects of hyaluronic acid"}

[TOOL RESULT]
Content for https://www.ncbi.nlm.nih.gov/books/NBK482440/: The most common adverse effects associated with hyaluronic acid filler are pain, bruising, redness, itching, and swelling . These side effects are self-limited and typically last no more than seven days.

Content for https://my.clevelandclinic.org/health/articles/22915-hyaluronic-acid: Research shows that hyaluronic acid is safe to use. Reactions or adverse effects from hyaluronic acid are rare, and it's safe to use if you're ...

Content for https://www.nebraskamed.com/health/head-and-neck/dermatology/hyaluronic-acid-what-is-it-what-it-treats-and-side-effects-to-be: There are no known side effects to topical lotions and creams with hyaluronic acid, as it's already present in your body. Still, you could have ...

Content for https://www.webmd.com/vitamins/ai/ingredientmono-1062/hyaluronic-acid: Hardening of skin and c

### 📄 Inspecting Saved Documents
The documents retrieved by the web tool were saved in the agent's internal state. Inspect them using the `documents` field of the output.

In [5]:
agent_results["documents"]

[Document(id=c4cf681884b82adb04a01af2ed26e71635a94542d8a9d1858517ceb73f84fcba, content: 'The most common adverse effects associated with hyaluronic acid filler are pain, bruising, redness, ...', meta: {'title': 'Hyaluronic Acid - StatPearls - NCBI Bookshelf', 'link': 'https://www.ncbi.nlm.nih.gov/books/NBK482440/', 'position': 1}),
 Document(id=1b91a8c533c478de952acdc66549221080d6319ad416cbb489b0335129e1395b, content: 'Research shows that hyaluronic acid is safe to use. Reactions or adverse effects from hyaluronic aci...', meta: {'title': 'Hyaluronic Acid: What It Is, Benefits, How To Use & Side Effects', 'link': 'https://my.clevelandclinic.org/health/articles/22915-hyaluronic-acid', 'date': 'May 4, 2022', 'position': 2}),
 Document(id=2c9b29d0d6a5e324a2e7ab522203ba5f71048958242811a16c9327dc37a7f398, content: 'There are no known side effects to topical lotions and creams with hyaluronic acid, as it's already ...', meta: {'title': 'Hyaluronic acid: What is it, what it treats, and side e